
# ViEWS 3 constituent models 
## ViEWS production system, cm level


This notebook trains a set of regression models for use in the monthly updated ViEWS predicting fatalities ensemble

The notebook does the following: 
1. Retrieves data through querysets and stores in DataSets, a list of dictionaries
2. Specifies the metadata of a number of models, stores in ModelList, a list of dictionaries
3. Trains the models in ModelList, stores the trained objects in model storage and prediction storage
4. Saves part of ModelList as csv and the rest as pickles


## Importing modules

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
# sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression

from xgboost import XGBRegressor
from xgboost import XGBClassifier
from xgboost import XGBRFRegressor, XGBRFClassifier

from lightgbm import LGBMClassifier, LGBMRegressor

# Views 3
from viewser.operations import fetch
import views_runs
from views_partitioning import data_partitioner, legacy
from stepshift import views
import views_dataviz
from views_runs import storage
from views_runs.storage import store, retrieve, fetch_metadata

from views_forecasts.extensions import *

# Other packages
import pickle as pkl

# Packages from viewsforecasting repository

#from Ensembling import CalibratePredictions, RetrieveStoredPredictions, mean_sd_calibrated, gam_calibrated
import os
import sys
sys.path.append('../')
sys.path.append('../Tools')
sys.path.append('../Intermediates')
from FetchData import FetchData, RetrieveFromList, document_queryset, ReturnQsList, document_ensemble
from ViewsEstimators import *


## Common parameters

In [3]:
#!conda list | grep views

In [4]:
# To do:
# find out why and where missingness occurs

In [5]:
# Common parameters:
dev_id = 'Fatalities002'
run_id = dev_id

# Generating a new run if necessary

#try:
#    ViewsMetadata().new_run(name=run_id,description='Developing the fatalities model for FCDO',min_month=1,max_month=999)
#except KeyError:
#    if 'devel' not in run_id:
#        warnings.warn('You are overwriting a production system')

RerunQuerysets = True

FutureStart = 508
steps = [*range(1, 36+1, 1)] # Which steps to train and predict for
fi_steps = [1,3,6,12,36] # Which steps to present feature importances for
#steps = [1,3,6,12,36]
#fi_steps = [1,3,6,12,36]

# Specifying partitions
calib_partitioner_dict = {"train":(121,396),"predict":(397,444)}
test_partitioner_dict = {"train":(121,444),"predict":(445,492)}
future_partitioner_dict = {"train":(121,492),"predict":(493,504)}
calib_partitioner =  views_runs.DataPartitioner({"calib":calib_partitioner_dict})
test_partitioner =  views_runs.DataPartitioner({"test":test_partitioner_dict})
future_partitioner =  views_runs.DataPartitioner({"future":future_partitioner_dict})

username = os.getlogin()
Mydropbox = f'/Users/{os.getlogin()}/Dropbox (ViEWS)/ViEWS'
overleafpath = f'/Users/{username}/Dropbox (ViEWS)/Apps/Overleaf/VIEWS documentation {dev_id}/'
print('Setting Mydropbox to',Mydropbox)

print('User:', username)
print('Overleaf path set to',overleafpath)

Setting Mydropbox to /Users/havardhegre1/Dropbox (ViEWS)/ViEWS
User: havardhegre1
Overleaf path set to /Users/havardhegre1/Dropbox (ViEWS)/Apps/Overleaf/VIEWS documentation Fatalities002/


# Retrieve data

In [6]:
# Create Markdown documentation of all querysets used
level = 'cm'
qslist = ReturnQsList(level)
document_queryset(qslist,dev_id)

 .    fatalities002_baseline; A dataset with 6 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_topics_stub; A dataset with 62 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_aquastat_stub; A dataset with 11 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_cm_conflict_history_stub; A dataset with 24 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_cm_conflict_history_ext; A dataset with 33 columns, with data between t = 1 and 852. (213 units)
 .    fatalities002_vdem_short_stub; A dataset with 57 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_wdi_short_stub; A dataset with 26 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_joint_narrow; A dataset with 39 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_joint_broad_stub; A dataset with 102 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_faostat_stub;A

In [7]:
from FetchData import fetch_cm_data_from_model_def

Datasets=fetch_cm_data_from_model_def(qslist)

 .    joint_broad: A dataset with 108 columns, with data between t = 1 and 852; 213 units.
 .    vdem_short: A dataset with 63 columns, with data between t = 1 and 852; 213 units.
 .    faostat: A dataset with 41 columns, with data between t = 1 and 852; 213 units.
 .    topics_002: A dataset with 68 columns, with data between t = 1 and 852; 213 units.
 .    baseline002: A dataset with 6 columns, with data between t = 1 and 852; 213 units.
 .    aquastat: A dataset with 17 columns, with data between t = 1 and 852; 213 units.
 .    wdi_short: A dataset with 32 columns, with data between t = 1 and 852; 213 units.
 .    all_features: A dataset with 186 columns, with data between t = 1 and 852; 213 units.
 .    conflictlong_ln: A dataset with 63 columns, with data between t = 1 and 852; 213 units.
 .    imfweo: A dataset with 11 columns, with data between t = 1 and 852; 213 units.
 .    faoprices: A dataset with 17 columns, with data between t = 1 and 852; 213 units.
 .    joint_narrow: A 

# Generating predictions
Using the ViEWS3 partitioning/stepshifting syntax. Training models for A: calibration partition and B: test partition, to test out some calibration routines. Most models trained with ln_ged_sb_best as outcome.

In [8]:
dev_id

'Fatalities002'

In [9]:
Datasets[0]['df'].head()

gleditsch_ward  ln_ged_sb_dep  ln_ged_sb  ln_ged_ns  \
month_id country_id                                                        
1        1                    110.0            0.0        0.0        0.0   
         2                    115.0            0.0        0.0        0.0   
         3                     52.0            0.0        0.0        0.0   
         4                    101.0            0.0        0.0        0.0   
         5                    990.0            0.0        0.0        0.0   

                     ln_ged_os  ln_acled_sb  ln_acled_sb_count  ln_acled_os  \
month_id country_id                                                           
1        1                 0.0          0.0                0.0          0.0   
         2                 0.0          0.0                0.0          0.0   
         3                 0.0          0.0                0.0          0.0   
         4                 0.0          0.0                0.0          0.0   
         5                 0.0          0.0                0.0          0.0   

                     wdi_sp_pop_totl  wdi_sm_pop_netm  ...  \
month_id country_id                                    ...   
1        1                  780153.0         -97340.0  ...   
         2                  359531.0         -31771.0  ...   
         3                 1084744.0         -33669.0  ...   
         4                15182611.0           1380.0  ...   
         5                  155525.0         -18695.0  ...   

                     topic10_conflict_t1_stock  topic11_diplomacy_t1_stock  \
month_id country_id                                                          
1        1                             0.14355                     2.76834   
         2                             0.00000                     0.00000   
         3                             2.78406                     9.27688   
         4                             1.71016                     3.68399   
         5                             0.00000                     0.00000   

                     topic12_power_t1_stock  topic13_sports_t1_stock  \
month_id country_id                                                    
1        1                          1.87957                60.850071   
         2                          0.00000                 0.000000   
         3                          4.89045                 0.854790   
         4                         11.73558                 3.056060   
         5                          0.00000                 0.000000   

                     topic14_judiciary_t1_stock  \
month_id country_id                               
1        1                              0.93191   
         2                              0.00000   
         3                              3.08684   
         4                              3.73062   
         5                              0.00000   

                     splag_topic2_sanctions_t1_stock  \
month_id country_id                                    
1        1                                   1.32903   
         2                                   1.19587   
         3                                   0.00000   
         4                                   2.38782   
         5                                   0.00000   

                     splag_topic10_conflict_t1_stock  \
month_id country_id                                    
1        1                                   6.36659   
         2                                   4.79998   
         3                                   0.00000   
         4                                  19.63530   
         5                                   0.00000   

                     splag_topic11_diplomacy_t1_stock  \
month_id country_id                                     
1        1                                    6.56443   
         2                                    5.64878   
         3                                    0.00000   
         4        

In [10]:
from views_runs import ModelMetadata 
help(ModelMetadata)

Help on class ModelMetadata in module views_schema.models:

class ModelMetadata(pydantic.main.BaseModel)
 |  ModelMetadata(*, author: str, queryset_name: str, train_start: int, train_end: int, steps: List[int] = None, training_date: datetime.datetime) -> None
 |  
 |  ModelMetadata
 |  =============
 |  
 |  Data used to organize model objects.
 |  
 |  parameters:
 |      author (str): Name of the user that authored the model object.
 |      queryset_name (str): Name of the queryset used to train the model
 |      train_start (int): Month identifier for training start date
 |      train_start (int): Month identifier for training end date
 |      training_date (datetime.datetime): Timestamp for training date (use datetime.datetime.now())
 |  
 |  example:
 |  
 |      # Instantiate the class with values
 |  
 |      my_metadata = ModelMetadata(
 |          author = "my_name",
 |          queryset_name = "my_queryset",
 |          train_start = 1,
 |          train_end = 300,
 |        

## Checking missingness and infinity values

In [11]:
N=51
for i in range(len(Datasets)):
    df = Datasets[i]['df']
    print(Datasets[i]['Name'])
    for col in df.iloc[: , :N].columns:
        print(col,len(df[col]), 'missing:', df[col].isnull().sum(), 'infinity:', np.isinf(df).values.sum())


joint_broad
gleditsch_ward 158230 missing: 0 infinity: 0
ln_ged_sb_dep 158230 missing: 0 infinity: 0
ln_ged_sb 158230 missing: 0 infinity: 0
ln_ged_ns 158230 missing: 0 infinity: 0
ln_ged_os 158230 missing: 0 infinity: 0
ln_acled_sb 158230 missing: 1437 infinity: 0
ln_acled_sb_count 158230 missing: 1437 infinity: 0
ln_acled_os 158230 missing: 1437 infinity: 0
wdi_sp_pop_totl 158230 missing: 11 infinity: 0
wdi_sm_pop_netm 158230 missing: 11 infinity: 0
wdi_sm_pop_refg_or 158230 missing: 11 infinity: 0
wdi_dt_oda_odat_pc_zs 158230 missing: 11 infinity: 0
wdi_ms_mil_xpnd_gd_zs 158230 missing: 11 infinity: 0
wdi_sl_tlf_totl_fe_zs 158230 missing: 11 infinity: 0
wdi_nv_agr_totl_kn 158230 missing: 11 infinity: 0
wdi_sp_pop_grow 158230 missing: 11 infinity: 0
wdi_se_enr_prim_fm_zs 158230 missing: 11 infinity: 0
wdi_sp_urb_totl_in_zs 158230 missing: 11 infinity: 0
wdi_sh_sta_maln_zs 158230 missing: 11 infinity: 0
wdi_sp_dyn_imrt_fe_in 158230 missing: 11 infinity: 0
wdi_ny_gdp_mktp_kd 158230 mis

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 11 infinity: 0
vdem_v2x_libdem_48 158230 missing: 32 infinity: 0
vdem_v2x_partip 158230 missing: 11 infinity: 0
vdem_v2x_partipdem 158230 missing: 11 infinity: 0
vdem_v2x_accountability 158230 missing: 11 infinity: 0
vdem_v2x_clphy 158230 missing: 11 infinity: 0
vdem_v2x_cspart 158230 missing: 11 infinity: 0
vdem_v2x_divparctrl 158230 missing: 11 infinity: 0
vdem_v2x_edcomp_thick 158230 missing: 11 infinity: 0
vdem_v2x_egal 158230 missing: 11 infinity: 0
vdem_v2x_execorr 158230 missing: 11 infinity: 0
vdem_v2x_frassoc_thick 158230 missing: 11 infinity: 0
vdem_v2x_gencs 158230 missing: 11 infinity: 0
vdem_v2x_gender 158230 missing: 11 infinity: 0
vdem_v2x_genpp 158230 missing: 11 infinity: 0
vdem_v2x_horacc 158230 missing: 11 infinity: 0
vdem_v2x_neopat 158230 missing: 11 infinity: 0
vdem_v2x_pubcorr 158230 missing: 11 infinity: 0
conflictlong_ln
gleditsch_ward 158230 missing: 0 infinity: 0
ln_ged_sb_dep 158230 missing: 0 infinity: 0
ln_ged_sb 158230 missing: 0 infinity: 0
ln_ged_ns 15

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Specify models in ensemble

In [ ]:
from ModelDefinitions import DefineEnsembleModels

ModelList = DefineEnsembleModels('cm')
    
for imodel,model in enumerate(ModelList):
    print(imodel, model['modelname'], model['data_train'])

In [ ]:
ModelList

In [ ]:
outcome = 'sb'
EnsembleMetaData_df = document_ensemble(ModelList,outcome)
if username == 'havardhegre1':
    filename = overleafpath + f'Tables/Evaluation/Ensemble_{outcome}.md'
    EnsembleMetaData_df.to_markdown(index=False, buf=filename)

In [ ]:
# Loop that checks whether the model exists, retrains if not, 
# and stores the predictions if they have not been stored before for this run.
# To do: set the data_preprocessing to the function in the model dictionary

level = 'cm'
includeFuture = False

from views_runs import Storage, StepshiftedModels
from views_partitioning.data_partitioner import DataPartitioner
from viewser import Queryset, Column
from views_runs import operations
from views_runs.run_result import RunResult

i = 0
for model in ModelList:
    if model['algorithm'] != 'Rscript':
        force_retrain = False
        modelstore = storage.Storage()
        ct = datetime.now()
        print(i, model['modelname'])
        print('Calibration partition', ct)
        model['Algorithm_text'] = str(model['algorithm'])
        model['RunResult_calib'] = RunResult.retrain_or_retrieve(
                retrain            = force_retrain,
                store              = modelstore,
                partitioner        = DataPartitioner({"calib":calib_partitioner_dict}),
                stepshifted_models = StepshiftedModels(model['algorithm'], steps, model['depvar']),
                dataset            = RetrieveFromList(Datasets,model['data_train']),
                queryset_name      = model['queryset'],
                partition_name     = "calib",
                timespan_name      = "train",
                storage_name       = model['modelname'] + '_calib',
                author_name        = "HH",
        )

    #    model['predstore_calib'] = level +  '_' + model['modelname'] + '_calib'
        ct = datetime.now()
        print('Trying to retrieve predictions', ct)
        try:
            predictions_calib = pd.DataFrame.forecasts.read_store(run=run_id, name=model['predstore_calib'])
        except KeyError:
            print(model['predstore_calib'], ', run',  run_id, 'does not exist, predicting')
            predictions_calib = model['RunResult_calib'].run.predict("calib","predict", model['RunResult_calib'].data)
            predictions_calib.forecasts.set_run(run_id)
            predictions_calib.forecasts.to_store(name=model['predstore_calib'])

        ct = datetime.now()
        print('Test partition', ct)
        modelstore = storage.Storage()
        model['RunResult_test'] = RunResult.retrain_or_retrieve(
                retrain            = force_retrain,
                store              = modelstore,
                partitioner        = DataPartitioner({"test":test_partitioner_dict}),
                stepshifted_models = StepshiftedModels(model['algorithm'], steps, model['depvar']),
                dataset            = RetrieveFromList(Datasets,model['data_train']),
                queryset_name      = model['queryset'],
                partition_name     = "test",
                timespan_name      = "train",
                storage_name       = model['modelname'] + '_test',
                author_name        = "HH",
        )
        ct = datetime.now()
        print('Trying to retrieve predictions', ct)
    #    model['predstore_test'] = level +  '_' + model['modelname'] + '_test'
        try:
            predictions_test = pd.DataFrame.forecasts.read_store(run=run_id, name=model['predstore_test'])
        except KeyError:
            print(model['predstore_test'], ', run', run_id, 'does not exist, predicting')
            predictions_test = model['RunResult_test'].run.predict("test","predict",model['RunResult_test'].data)
            predictions_test.forecasts.set_run(run_id)
            predictions_test.forecasts.to_store(name=model['predstore_test'])
        # Predictions for true future
        if includeFuture:
            ct = datetime.now()
            print('Future', ct)
            modelstore = storage.Storage()
            model['RunResult_future'] = RunResult.retrain_or_retrieve(
                    retrain            = force_retrain,
                    store              = modelstore,
                    partitioner        = DataPartitioner({"test":future_partitioner_dict}),
                    stepshifted_models = StepshiftedModels(model['algorithm'], steps, model['depvar']),
                    dataset            = RetrieveFromList(Datasets,model['data_train']),
                    queryset_name      = model['queryset'],
                    partition_name     = "test",
                    timespan_name      = "train",
                    storage_name       = model['modelname'] + '_future',
                    author_name        = "HH",
            )
            ct = datetime.now()
            print('Trying to retrieve predictions', ct)
            model['predstore_future'] = level +  '_' + model['modelname'] + '_f' + str(FutureStart)
            try:
                predictions_future = pd.DataFrame.forecasts.read_store(run=run_id, name=model['predstore_future'])
            except KeyError:
                print(model['predstore_future'], ', run', run_id, 'does not exist, predicting')
                predictions_future = model['RunResult_future'].run.future_point_predict(FutureStart,model['RunResult_future'].data)
                predictions_future.forecasts.set_run(run_id)
                predictions_future.forecasts.to_store(name=model['predstore_future'])  
        print('**************************************************************')
    i = i + 1

print('All done')

In [ ]:
# Exploring the future predictions


predictions_test.xs(246,level=1).tail()

## Notes on training time for the various algorithms:

In [ ]:
#These are calculated in minutes for the hh20 feature set (with about 40 features), for all 36 steps, calibration (c) and test (t) partitions, also include generating predictions, and are approximate:

#nj=12 (number of threads)
#scikit random forest:        21:13 (c), 26:20 (t) RandomForestRegressor(n_estimators=200, n_jobs=nj)
#XGB random forest:           06:02 (c), 07:51 (t) XGBRFRegressor(n_estimators=300,n_jobs=nj)
#scikit gbm:                  13:59 (c), 15:55 (t) GradientBoostingRegressor(), 
#scikit hurdle random forest: 07:32 (c), 09:49 (t) For both clf and reg: (n_estimators=200, n_jobs=nj)
#XGB hurdle xgb:              01:26 (c), 01:32 (t) For both clf and reg:                n_estimators=200,tree_method='hist',n_jobs=nj)
#scikit histgbm:              01:17 (c), 01:20 (t) HistGradientBoostingRegressor(max_iter=200)
#XGB xgb:                     01:00 (c), 01:04 (t) XGBRegressor(n_estimators=200,tree_method='hist',n_jobs=nj)
#lightgbm gbm:                00:25 (c), --    (t) LGBMRegressor(n_estimators=100,num_threads=8)

# Various helper functions and tools....

In [ ]:
!conda list | grep views-forecasts

# Retrieving external forecasts

In [ ]:
# Retrieve David's Markov models
# To do: rewrite the model dictionary to the new, slimmer version.
DRList = []


model = {
    'modelname':   'fat_hh20_Markov_glm',
    'algorithm': [],
    'depvar': "ln_ged_sb_dep",
    'data_train':      'hh20',
    'queryset': 'hh_20_features',
}
DRList.append(model)

model = {
    'modelname':   'fat_hh20_Markov_rf',
    'algorithm': [],
    'depvar': "ln_ged_sb_dep",
    'data_train':      'hh20',
    'queryset': 'hh_20_features',
}

DRList.append(model)



In [ ]:
path = f'/Users/{os.getlogin()}/Dropbox (ViEWS)/ViEWS/Projects/PredictingFatalities/Predictions/cm/preds/'

DRList[0]['predictions_file_calib'] = path + 'vmm_glm_hh20_0125_alt_calib.csv'
DRList[0]['predictions_file_test'] = path + 'vmm_glm_hh20_0125_alt_test.csv'
DRList[0]['predictions_file_future'] = path + 'vmm_glm_hh20_506.csv'

DRList[1]['predictions_file_calib'] = path + 'vmm_rf_hh20_0125_alt_calib.csv'
DRList[1]['predictions_file_test'] = path + 'vmm_rf_hh20_0125_alt_test.csv'
DRList[1]['predictions_file_future'] = path + 'vmm_rf_hh20_505.csv'

In [ ]:
print(path)

In [ ]:

for model in ModelList:
    print(model['modelname'])

In [ ]:
# Storing Markov models in central storage
# Retrieving dependent variable
target_calib = pd.DataFrame.forecasts.read_store('cm_fat_conflicthistory_rf_calib', run=run_id)['ln_ged_sb_dep']
target_test = pd.DataFrame.forecasts.read_store('cm_fat_conflicthistory_rf_test', run=run_id)['ln_ged_sb_dep']
level = 'cm'
for model in DRList:
    df_calib = pd.read_csv(model['predictions_file_calib'],index_col=['month_id','country_id'])
    df_test = pd.read_csv(model['predictions_file_test'],index_col=['month_id','country_id'])
    df_future = pd.read_csv(model['predictions_file_future'],index_col=['month_id','country_id'])
    df_calib['ln_ged_sb_dep'] = target_calib
    df_test['ln_ged_sb_dep'] = target_test
    df_future['ln_ged_sb_dep'] = np.nan # Empty dependent variable column for consistency/required by prediction storage function
    stored_modelname = level + '_' + model['modelname'] + '_calib'
    df_calib.forecasts.set_run(run_id)
    df_calib.forecasts.to_store(name=stored_modelname, overwrite=True)
    stored_modelname = level + '_' + model['modelname'] + '_test'
    df_test.forecasts.set_run(run_id)
    df_test.forecasts.to_store(name=stored_modelname, overwrite=True)    

In [ ]:
ModelList